## Importando librerías

In [1]:
import pandas as pd
import numpy as np
import string
import re

In [2]:
import os

## Llevando texto a dataframe

### Cargando markdowns

In [3]:
# Get the list of all files and directories
path = "Markdown/"
filelist = os.listdir(path)

In [4]:
filelist.sort()

In [5]:
filelist

['.ipynb_checkpoints',
 '00 - Preámbulo.md',
 '01 - Principios y Disposiciones Generales.md',
 '02 - Derechos Fundamentales y Garantías.md',
 '03 - Naturaleza y Medioambiente.md',
 '04 - Participación Democrática.md',
 '05 - Buen Gobierno y Función Pública.md',
 '06 - Estado Regional y Organización Territorial.md',
 '07 - Poder Legislativo.md',
 '08 - Poder Ejecutivo.md',
 '09 - Sistemas de Justicia.md',
 '10 - Órganos Autónomos Constitucionales.md',
 '11 - Reforma y Reemplazo de la Constitución.md',
 '12 - Disposiciones Transitorias.md',
 '13 - Convención Constitucional.md']

In [6]:
df_constitucion = pd.DataFrame()
texto_total = ''

for file in filelist[1:]:
    with open(path+file, 'r', encoding = 'utf-8') as f:
        text = f.read()
        if text[-1:] != '\n':
            text = text+'\n'
    
    texto_total = texto_total + text

serie = texto_total.split('\n')

### Llevando a dataframe

In [7]:
df = pd.DataFrame({'texto':serie})

df['capitulo'] = np.where(df['texto'].str.contains('# Preámbulo'),df['texto'],np.nan)
df['capitulo'] = np.where(df['texto'].str.contains('# Disposiciones Transitorias'),df['texto'],df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('# Convención Constitucional'),df['texto'],df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Capítulo'),df['texto'],df['capitulo'])

#Para asegurar que aparezca el capitulo IV
df['capitulo'] = np.where(df['texto'].str.contains('Capitulo'),df['texto'],df['capitulo'])
df['articulo'] = np.where(df['texto'].str.contains('### Artículo'),df['texto'],np.nan)
df['texto'] = np.where(df['texto']=='', np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['articulo'], np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['capitulo'], np.nan, df['texto'])
# Agregar numeral y ordinal


df['capitulo'].fillna(method='pad', inplace=True)
df['articulo'].fillna(method='pad', inplace=True)


In [8]:
df.dropna(subset=['texto'], inplace = True)

In [9]:
df['nro_capitulo'] = df['capitulo'].str.split(':', expand = True)[0]
df['nro_capitulo'] = df['nro_capitulo'].str.replace('# ','')
df['titulo_capitulo'] = df['capitulo'].str.split(':', expand = True)[1]
df['titulo_capitulo'] = df['titulo_capitulo'].str.strip()
df['articulo'] = df['articulo'].str.replace('### ','')

In [10]:
df['articulo'] = np.where(df['nro_capitulo']=='Convención Constitucional',np.nan,df['articulo'])

In [11]:
df

,texto,capitulo,articulo,nro_capitulo,titulo_capitulo
2,"Nosotras y nosotros, el pueblo de Chile, confo...",# Preámbulo,NaN,Preámbulo,None
7,1. Chile es un Estado social y democrático de ...,# Capítulo I: Principios y Disposiciones Gener...,Artículo 1,Capítulo I,Principios y Disposiciones Generales
8,2. Se constituye como una república solidaria....,# Capítulo I: Principios y Disposiciones Gener...,Artículo 1,Capítulo I,Principios y Disposiciones Generales
9,3. La protección y garantía de los derechos hu...,# Capítulo I: Principios y Disposiciones Gener...,Artículo 1,Capítulo I,Principios y Disposiciones Generales
13,"1. La soberanía reside en el pueblo de Chile, ...",# Capítulo I: Principios y Disposiciones Gener...,Artículo 2,Capítulo I,Principios y Disposiciones Generales
...,...,...,...,...,...
3188,Carolina Vilches Fuenzalida,# Convención Constitucional,NaN,Convención Constitucional,None
3190,Ingrid Villena Narbona,# Convención Constitucional,NaN,Convención Constitucional,None
3192,Manuel Woldarsky González,# Convención Constitucional,NaN,Convención Constitucional,None
3194,Camila Zárate Zárate,# Convención Constitucional,NaN,Convención Constitucional,None


In [12]:
df = df[['nro_capitulo','titulo_capitulo','articulo','texto']]
df.rename(columns = {'nro_capitulo':'capitulo'}, inplace = True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
df.titulo_capitulo.value_counts()

Derechos Fundamentales y Garantías            354
Estado Regional y Organización Territorial    232
Poder Legislativo                             194
Órganos Autónomos Constitucionales            140
Sistemas de Justicia                          121
Poder Ejecutivo                               114
Naturaleza y Medioambiente                     66
Buen Gobierno y Función Pública                57
Participación Democrática                      41
Principios y Disposiciones Generales           32
Reforma y Reemplazo de la Constitución         26
Name: titulo_capitulo, dtype: int64

In [14]:
import googletrans
from googletrans import Translator

In [15]:
translator = Translator(service_urls=['translate.googleapis.com'])
translated = translator.translate('Hola Mundo', dest='en')

# obtain translated text - attribute .text
print(translated)


Translated(src=es, dest=en, text=Hello World, pronunciation=Hello World, extra_data="{'translat...")


In [16]:
df['texto_en'] = ''
df

,capitulo,titulo_capitulo,articulo,texto,texto_en
2,Preámbulo,None,NaN,"Nosotras y nosotros, el pueblo de Chile, confo...",
7,Capítulo I,Principios y Disposiciones Generales,Artículo 1,1. Chile es un Estado social y democrático de ...,
8,Capítulo I,Principios y Disposiciones Generales,Artículo 1,2. Se constituye como una república solidaria....,
9,Capítulo I,Principios y Disposiciones Generales,Artículo 1,3. La protección y garantía de los derechos hu...,
13,Capítulo I,Principios y Disposiciones Generales,Artículo 2,"1. La soberanía reside en el pueblo de Chile, ...",
...,...,...,...,...,...
3188,Convención Constitucional,None,NaN,Carolina Vilches Fuenzalida,
3190,Convención Constitucional,None,NaN,Ingrid Villena Narbona,
3192,Convención Constitucional,None,NaN,Manuel Woldarsky González,
3194,Convención Constitucional,None,NaN,Camila Zárate Zárate,


In [17]:
i = 0
for index, row in df.iterrows():
    row['texto_en'] = translator.translate(row['texto']).text

In [18]:
df

,capitulo,titulo_capitulo,articulo,texto,texto_en
2,Preámbulo,None,NaN,"Nosotras y nosotros, el pueblo de Chile, confo...","We, the people of Chile, made up of various na..."
7,Capítulo I,Principios y Disposiciones Generales,Artículo 1,1. Chile es un Estado social y democrático de ...,1. Chile is a social and democratic state of l...
8,Capítulo I,Principios y Disposiciones Generales,Artículo 1,2. Se constituye como una república solidaria....,2. It is constituted as a solidary republic. I...
9,Capítulo I,Principios y Disposiciones Generales,Artículo 1,3. La protección y garantía de los derechos hu...,3. The protection and guarantee of individual ...
13,Capítulo I,Principios y Disposiciones Generales,Artículo 2,"1. La soberanía reside en el pueblo de Chile, ...","1. Sovereignty resides in the people of Chile,..."
...,...,...,...,...,...
3188,Convención Constitucional,None,NaN,Carolina Vilches Fuenzalida,Carolina Vilches Fuenzalida
3190,Convención Constitucional,None,NaN,Ingrid Villena Narbona,Ingrid Villena Narbonne
3192,Convención Constitucional,None,NaN,Manuel Woldarsky González,Manuel Woldarsky Gonzalez
3194,Convención Constitucional,None,NaN,Camila Zárate Zárate,Camila Zarate Zarate


In [19]:
f = open("nc.html", "w")

capitulo = ''
titulo = ''
articulo = ''

for index, row in df.iterrows():
    if row['capitulo'] != capitulo:
        capitulo = row['capitulo']
        f.write("<div class='row justify-content-center'><div class='col-10 col-md-8 col-lg-6'><h1>"+capitulo+'</h1></div></div>\n')

    if row['titulo_capitulo'] != titulo:
        titulo = row['titulo_capitulo']
        if titulo != None:
            f.write("<div class='row justify-content-center'><div class='col-10 col-md-8 col-lg-6'><h2>"+titulo+'</h2></div></div>\n')

    if row['articulo'] != articulo:
        articulo = row['articulo']
        if pd.isnull(row['articulo']) == False:
            f.write("<div class='row justify-content-center'><div class='col-10 col-md-8 col-lg-6'><h3>"+articulo+'</h3></div></div>\n')
    
    f.write("<div class='row justify-content-center'>")
    f.write("<div class='col-10 col-md-8 col-lg-6'>\n")
    f.write("<p class='article'>"+row['texto_en']+'</p>\n')
    f.write("<p class='article d-none'>"+row['texto']+'</p>\n')
    f.write('</div>\n')
    f.write('</div>\n')
f.close()